<a href="https://colab.research.google.com/github/yifan-07/110_covid-19-impact-on-digital-learning/blob/master/area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

選定範圍做地毯式搜索店家



In [ ]:
# %run file.py/.ipynb import fun
%run /content/drive/MyDrive/Program/google_map/python/nearbysearch.ipynb import get_geometries

In [ ]:
# import package
import geopandas as gpd
from google.colab import drive
import os
import pandas as pd
import folium
import shapely
from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon
import numpy as np

drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/Program/google_map/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# .shp, .dbf and .shx in the same file
path = '/content/drive/MyDrive/Program/google_map/data/'
county = gpd.read_file(path + 'address/polygon/COUNTY_MOI_1090820.shp', encoding = 'utf-8') # taiwan polygon
town = gpd.read_file(path + 'address/polygon/TOWN_MOI_1120317.shp', encoding = 'utf-8')

In [ ]:
dat1 = county[county['COUNTYNAME'] == '金門縣'].reset_index(drop = True) # Islands District
dat2 = county[county['COUNTYNAME'] == '連江縣'].reset_index(drop = True)
dat3 = county[county['COUNTYNAME'] == '澎湖縣'].reset_index(drop = True)
dat4 = county[~county['COUNTYNAME'].str.contains('金門|連江|澎湖', na = False)].reset_index(drop = True)

In [ ]:
for i in range(len(dat4)):
    if type(dat4['geometry'][i]) == shapely.geometry.multipolygon.MultiPolygon: #　contains island
        filtered_polygons = [polygon for polygon in dat4['geometry'][i].geoms if polygon.area * (111.32 ** 2) > 5] # convert km² and remove too small area
        filtered_multi_polygon = MultiPolygon(filtered_polygons)
        dat4['geometry'][i] = filtered_multi_polygon

# 缺
1. 區域分割

In [ ]:
[dat1_xmin, dat1_ymin, dat1_xmax, dat1_ymax] = dat1.total_bounds
[dat2_xmin, dat2_ymin, dat2_xmax, dat2_ymax] = dat2.total_bounds
[dat3_xmin, dat3_ymin, dat3_xmax, dat3_ymax] = dat3.total_bounds
[dat4_xmin, dat4_ymin, dat4_xmax, dat4_ymax] = dat4.total_bounds

polygons_ = []
for i in range(len(dat4)):
    polygons_.append(dat4['geometry'][i])
boundary = gpd.GeoSeries(unary_union(polygons_))

In [ ]:
data.columns

Index(['lat', 'lng', 'radius', 'left_bottom_point', 'right_top_point',
       'spacing', 'circle', 'circle_stores', 'stores', 'location'],
      dtype='object')

In [ ]:
data1 = get_geometries((dat1_xmin, dat1_ymin), (dat1_xmax, dat1_ymax))
data2 = get_geometries((dat2_xmin, dat2_ymin), (dat2_xmax, dat2_ymax))
data3 = get_geometries((dat3_xmin, dat3_ymin), (dat3_xmax, dat3_ymax))
data4 = get_geometries((dat4_xmin, dat4_ymin), (dat4_xmax, dat4_ymax))
data = pd.concat([data1, data2, data3, data4], ignore_index = True)

for i in data:
    if data['']

ValueError: ignored

In [ ]:
# plot area
map = folium.Map(location=[23.5250, 120.5823],  # start position
            width='100%', # windowd size
            height='100%',
            zoom_start=5, # initial zoom
            min_zoom=0, # min zoom
            max_zoom=18,
            tiles='OpenStreetMap', # map type
            attr="高德地图", # map source
            crs='EPSG3857' # 3857
            )
for dat in [dat1, dat2, dat3, dat4]:
    for _, r in dat.iterrows():
        # Without simplifying the representation of each borough,
        # the map might not be displayed
        sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'lineColor': '#ffffbf'})
        folium.Popup(r['COUNTYNAME']).add_to(geo_j)
        geo_j.add_to(map)


# for _, r in town.iterrows():
#     # Without simplifying the representation of each borough,
#     # the map might not be displayed
#     sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
#     geo_j = sim_geo.to_json()
#     geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'orange'})
#     folium.Popup(r['TOWNNAME']).add_to(geo_j)
#     geo_j.add_to(map)

map.add_child(folium.LatLngPopup()) # select area


In [ ]:
map = folium.Map(location=[23.5250, 120.5823],  # start position
                width='100%', # windowd size
                height='100%',
                zoom_start=5, # initial zoom
                min_zoom=0, # min zoom
                max_zoom=18,
                tiles='OpenStreetMap', # map type
                attr="高德地图", # map source
                crs='EPSG3857' # 3857
                )
for i in range(len(data)):
    x = data['lat'][i] # lat
    y = data['lng'][i] # lng
    radius = data['radius'][i]
    if type(x) == int:
        x = x.astype('float')
    if type(y) == int:
        y = y.astype('float')
    if type(radius) == np.int64:
        radius = radius.astype('float')
    folium.Circle(
        location = [x, y],
        radius = radius, # unit meter
        popup = str(x) +','+ str(y), #poly_record['circle_stores'][i]
        fill_opacity = 0.5
    ).add_to(map)

map.add_child(folium.LatLngPopup())       # 这个